In [1]:
# Import modules
import pandas as pd
import cleaning
import utils

In [2]:
# open elections.xlsx
elections = pd.read_excel('elections.xlsx')
# open elections_with_geocodes.xlsx
geocodes = pd.read_excel('elections_with_geocodes.xlsx')
# open elections_with_tracts.xlsx
tracts = pd.read_excel('elections_with_tracts.xlsx')

groups = ['DP02', 'DP03', 'DP05']
key = 'ad8851f3bf6aaf76923ec4119b6f714cdfaa87d9'

In [3]:
tracts_dict = {'geocode': [], 'year': []}

# for each row in elections
for index, row in elections.iterrows():
    # if District # is not nan
    if not pd.isna(row['District #']):
        # get row in tracts where filename is the same
        tract_row = tracts[tracts['filename'] == row['filename']]
        # make sure is not nan
        if not pd.isna(tract_row['tracts'].iloc[0]):
            new_tracts = tract_row['tracts'].iloc[0].split(', ')
            # add to tracts_df
            for new_tract in new_tracts:
                tracts_dict['geocode'].append(new_tract)
                tracts_dict['year'].append(row['year'])
            
# create tracts_df
tracts_df = pd.DataFrame(tracts_dict)

# remove duplicates
tracts_df = tracts_df.drop_duplicates()

# reset index on tracts_df
tracts_df = tracts_df.reset_index(drop=True)

In [4]:
acs_data = []

In [5]:
# for each row in tracts_df
for index, row in tracts_df.iterrows():
    if index < len(acs_data):
        continue

    if index % 10 == 0:
        print(f"{index} of {len(tracts_df)}")

    geocode = row['geocode'].split('_')
    state = geocode[0].zfill(2)
    county = geocode[1].zfill(3)
    tract = geocode[2].zfill(6)
    year = row['year']
    
    for_param = f'tract:{tract}&in=state:{state}+county:{county}'
    data_dict = utils.query(year, groups, for_param, key)
    data_dict['geocode'] = row['geocode']
    data_dict['year'] = year

    acs_data.append(data_dict)

0 of 6410
10 of 6410
20 of 6410
30 of 6410


KeyboardInterrupt: 

In [ ]:
df = cleaning.dicts_to_df(acs_data)

# order so that geocode and year are first
df = df[['geocode', 'year'] + [col for col in df.columns if col not in ['geocode', 'year']]]

# save as csv
# df.to_csv('tract_acs_data.csv', index=False)